In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ratings_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine_learning/user_games_data.csv')

In [3]:
ratings_data

,id,user_id,appid,playtime_forever
0,1,2,10,760
1,2,2,20,16
2,3,2,30,19
3,4,2,40,0
4,5,2,50,651
...,...,...,...,...
4152246,4152247,11868,250460,0
4152247,4152248,11868,250760,101
4152248,4152249,11868,252110,0
4152249,4152250,11868,252410,780


In [4]:
ratings_data = ratings_data[ratings_data['playtime_forever'] != 0]

In [5]:

# Отримання унікальних user_id
unique_user_ids = ratings_data['user_id'].unique()

# Обмеження кількості унікальних user_id до тисячі
limited_unique_user_ids = unique_user_ids[:8000].tolist()

# Фільтрація даних за обмеженими унікальними user_id
ratings_data = ratings_data[ratings_data['user_id'].isin(limited_unique_user_ids)]
# Результат: датафрейм з обмеженою кількістю унікальних user_id
print(ratings_data)


              id  user_id    appid  playtime_forever
0              1        2       10               760
1              2        2       20                16
2              3        2       30                19
4              5        2       50               651
5              6        2       60                38
...          ...      ...      ...               ...
3233580  3233581     8984  1426210               975
3233581  3233582     8984  1568590               948
3233585  3233586     8984   578080             10092
3233586  3233587     8984  1955960                59
3233589  3233590     8984  2119490                56

[1635212 rows x 4 columns]


In [6]:
len(ratings_data)

1635212

In [7]:
ratings_data.head()

,id,user_id,appid,playtime_forever
0,1,2,10,760
1,2,2,20,16
2,3,2,30,19
4,5,2,50,651
5,6,2,60,38


In [8]:
# Створення функції для створення рейтингу ігор для кожного користувача
def create_game_ratings(user_data):
    num_games = len(user_data)
    if num_games >= 5:
        user_data = user_data.sort_values(by='playtime_forever', ascending=False)
        user_data['rating'] = pd.qcut(user_data['playtime_forever'], q=5, labels=False, duplicates='drop') + 1
    else:
        user_data['rating'] = list(range(5, 5 - num_games, -1))
    return user_data

# Застосування функції create_game_ratings для кожного користувача
ratings_data = ratings_data.groupby('user_id').apply(create_game_ratings)

# Виведення результатів
print(ratings_data[['user_id', 'appid','rating']])

                 user_id    appid  rating
user_id                                  
2       487            2   271590     5.0
        505            2   377160     5.0
        499            2   365590     5.0
        581            2  1174180     5.0
        503            2   292030     5.0
...                  ...      ...     ...
8984    3233520     8984   608800     1.0
        3233476     8984    50620     1.0
        3233468     8984   355180     1.0
        3233457     8984   242590     1.0
        3233451     8984    20900     1.0

[1635212 rows x 3 columns]


In [9]:
print(ratings_data['rating'].unique())

[ 5.  4.  3.  2.  1. nan]


In [10]:
# Видалення рядків зі значеннями NaN у стовпці 'rating'
ratings_data = ratings_data.dropna(subset=['rating'])

# Виведення унікальних значень після видалення NaN
print(ratings_data['rating'].unique())


[5. 4. 3. 2. 1.]


In [11]:
# Видалення стовпця з назвою 'column_name'
ratings_data = ratings_data.drop('playtime_forever', axis=1)

In [12]:
# Виведення даних конкретного користувача (наприклад, користувача з ID = 2)
user_id_to_display = 2
user_data = ratings_data[ratings_data['user_id'] == user_id_to_display]
print(user_data[['user_id', 'appid', 'rating']])


             user_id    appid  rating
user_id                              
2       487        2   271590     5.0
        505        2   377160     5.0
        499        2   365590     5.0
        581        2  1174180     5.0
        503        2   292030     5.0
...              ...      ...     ...
        506        2   380600     1.0
        471        2     9880     1.0
        50         2     7800     1.0
        485        2   341190     1.0
        94         2    12130     1.0

[417 rows x 3 columns]


In [13]:
ratings_data = ratings_data.rename(columns={
    'user_id': 'userId',
    'appid': 'gameId',
})

In [14]:
ratings_data

id  userId   gameId  rating
user_id                                          
2       487          488       2   271590     5.0
        505          506       2   377160     5.0
        499          500       2   365590     5.0
        581          582       2  1174180     5.0
        503          504       2   292030     5.0
...                  ...     ...      ...     ...
8984    3233520  3233521    8984   608800     1.0
        3233476  3233477    8984    50620     1.0
        3233468  3233469    8984   355180     1.0
        3233457  3233458    8984   242590     1.0
        3233451  3233452    8984    20900     1.0

[1635183 rows x 4 columns]

In [15]:
game_metadata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine_learning/games_140k.csv')
game_metadata.head()

,steam_appid,type,name,genres
0,1466430,game,Aardvark,"[{'id': '37', 'name': 'Free to Play'}, {'id': ..."
1,1466460,dlc,Anime Artist 3 - A better Artist,"[{'id': '4', 'name': 'Casual'}, {'id': '23', '..."
2,1466470,dlc,Anime Artist 3 - Extended Wardrobe,"[{'id': '4', 'name': 'Casual'}, {'id': '23', '..."
3,1466480,dlc,Anime Artist 3 - Lewd Patch,"[{'id': '4', 'name': 'Casual'}, {'id': '23', '..."
4,1466490,game,TOKYO LOVE +,"[{'id': '25', 'name': 'Adventure'}]"


In [16]:
len(game_metadata)

145019

In [17]:
game_metadata = game_metadata[game_metadata['type'] == 'game']

# Виведення перших рядків для перевірки
print(game_metadata.head())

   steam_appid  type                            name  \
0      1466430  game                        Aardvark   
4      1466490  game                    TOKYO LOVE +   
5      1466500  game  AKUMA: Cyberpunk Rhythm Action   
6      1466520  game             Omen Exitio: Hunger   
7      1466530  game               Escape from Covid   

                                              genres  
0  [{'id': '37', 'name': 'Free to Play'}, {'id': ...  
4                [{'id': '25', 'name': 'Adventure'}]  
5                    [{'id': '1', 'name': 'Action'}]  
6  [{'id': '25', 'name': 'Adventure'}, {'id': '23...  
7  [{'id': '25', 'name': 'Adventure'}, {'id': '4'...  


In [18]:
len(game_metadata)

95735

In [19]:
game_metadata = game_metadata.rename(columns={
    'steam_appid': 'gameId',
    'name': 'title',
})


In [20]:
game = game_metadata
rating= ratings_data

In [21]:
game_metadata.head()

,gameId,type,title,genres
0,1466430,game,Aardvark,"[{'id': '37', 'name': 'Free to Play'}, {'id': ..."
4,1466490,game,TOKYO LOVE +,"[{'id': '25', 'name': 'Adventure'}]"
5,1466500,game,AKUMA: Cyberpunk Rhythm Action,"[{'id': '1', 'name': 'Action'}]"
6,1466520,game,Omen Exitio: Hunger,"[{'id': '25', 'name': 'Adventure'}, {'id': '23..."
7,1466530,game,Escape from Covid,"[{'id': '25', 'name': 'Adventure'}, {'id': '4'..."


In [22]:
game_data = pd.merge(ratings_data, game_metadata[['gameId', 'title', 'genres']], on='gameId', how='left')

# Виведемо результат


game_data.head()

,id,userId,gameId,rating,title,genres
0,488,2,271590,5.0,Grand Theft Auto V,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
1,506,2,377160,5.0,Fallout 4,"[{'id': '3', 'name': 'RPG'}]"
2,500,2,365590,5.0,Tom Clancy’s The Division™,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
3,582,2,1174180,5.0,Red Dead Redemption 2,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
4,504,2,292030,5.0,Відьмак® 3: Дикий гін,"[{'id': '3', 'name': 'Рольові ігри'}]"


In [23]:
# game_data.drop(columns=['genres'], inplace=True)

In [24]:
# game.drop(columns=['type'], inplace=True)

In [25]:
!pip install fuzzywuzzy

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [27]:
game_data.head()

,id,userId,gameId,rating,title,genres
0,488,2,271590,5.0,Grand Theft Auto V,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
1,506,2,377160,5.0,Fallout 4,"[{'id': '3', 'name': 'RPG'}]"
2,500,2,365590,5.0,Tom Clancy’s The Division™,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
3,582,2,1174180,5.0,Red Dead Redemption 2,"[{'id': '1', 'name': 'Action'}, {'id': '25', '..."
4,504,2,292030,5.0,Відьмак® 3: Дикий гін,"[{'id': '3', 'name': 'Рольові ігри'}]"


In [28]:
# Реіндексація для userId
game_data['new_userId'] = game_data.groupby('userId').ngroup() + 1

# Реіндексація для gameId
game_data['new_gameId'] = game_data.groupby('gameId').ngroup() + 1

# Виведення перших рядків для перевірки
print(game_data.head())


    id  userId   gameId  rating                       title  \
0  488       2   271590     5.0          Grand Theft Auto V   
1  506       2   377160     5.0                   Fallout 4   
2  500       2   365590     5.0  Tom Clancy’s The Division™   
3  582       2  1174180     5.0       Red Dead Redemption 2   
4  504       2   292030     5.0       Відьмак® 3: Дикий гін   

                                              genres  new_userId  new_gameId  
0  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1        3297  
1                       [{'id': '3', 'name': 'RPG'}]           1        6846  
2  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1        6409  
3  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1       22438  
4              [{'id': '3', 'name': 'Рольові ігри'}]           1        3831  


In [29]:

game_data.drop(columns=['userId'], inplace=True)
game_data = game_data.rename(columns={
    'gameId': 'steam_gameId',
})

In [30]:
print(game_data.head())

    id  steam_gameId  rating                       title  \
0  488        271590     5.0          Grand Theft Auto V   
1  506        377160     5.0                   Fallout 4   
2  500        365590     5.0  Tom Clancy’s The Division™   
3  582       1174180     5.0       Red Dead Redemption 2   
4  504        292030     5.0       Відьмак® 3: Дикий гін   

                                              genres  new_userId  new_gameId  
0  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1        3297  
1                       [{'id': '3', 'name': 'RPG'}]           1        6846  
2  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1        6409  
3  [{'id': '1', 'name': 'Action'}, {'id': '25', '...           1       22438  
4              [{'id': '3', 'name': 'Рольові ігри'}]           1        3831  


In [31]:
game_data = game_data.rename(columns={
    'new_gameId': 'gameId',
    'new_userId': 'userId',
})

In [32]:
print(game_data.head())

    id  steam_gameId  rating                       title  \
0  488        271590     5.0          Grand Theft Auto V   
1  506        377160     5.0                   Fallout 4   
2  500        365590     5.0  Tom Clancy’s The Division™   
3  582       1174180     5.0       Red Dead Redemption 2   
4  504        292030     5.0       Відьмак® 3: Дикий гін   

                                              genres  userId  gameId  
0  [{'id': '1', 'name': 'Action'}, {'id': '25', '...       1    3297  
1                       [{'id': '3', 'name': 'RPG'}]       1    6846  
2  [{'id': '1', 'name': 'Action'}, {'id': '25', '...       1    6409  
3  [{'id': '1', 'name': 'Action'}, {'id': '25', '...       1   22438  
4              [{'id': '3', 'name': 'Рольові ігри'}]       1    3831  


In [33]:
len(game_data)
len(game_data["gameId"].unique())

28786

In [34]:
unique_values = game_data[['gameId', 'steam_gameId', 'title','genres']].drop_duplicates()

# Збереження унікальних значень у новий датафрейм або файл
# Якщо потрібно зберегти у файл CSV:
unique_values.to_csv('unique_values_2.csv', index=False)

In [50]:
pivot_table = game_data.pivot_table(columns="userId", index="gameId", values="rating")
# pivot_table = game_data.pivot_table(index="userId", columns="gameId", values="rating")
pivot_table.shape

(28786, 7999)

In [51]:


pivot_table.head(5)



userId,1,2,3,4,5,6,7,8,9,10,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
gameId,,,,,,,,,,,,,,,,,,,,,
1,4.0,2.0,NaN,NaN,1.0,1.0,4.0,NaN,4.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,NaN,3.0,1.0,1.0,1.0,5.0,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

pivot_table.fillna(0, inplace=True)
pivot_table = pivot_table.astype(bool).astype(int)
similarity_table = cosine_similarity(pivot_table)
similarity_table.shape

In [ ]:


pov_userid = 1
similarity_table[pov_userid].argsort()[-5:][::-1]



In [ ]:


game_data[game_data["userId"] == 365].sort_values("rating", ascending=False).head(5)



In [40]:
knn = NearestNeighbors(metric="cosine")
knn.fit(pivot_table.values)

NearestNeighbors(metric='cosine')

In [41]:
knn.kneighbors(pivot_table.iloc[[pov_userid]])

(array([[2.22044605e-16, 6.11933525e-01, 6.18900104e-01, 6.29956888e-01,
         6.32298398e-01]]),
 array([[   1, 3762, 5173, 5761, 7133]]))

In [42]:
def get_gameId_by_title(title, game_data):
    game_ids = game_data[game_data['title'].str.contains(title, case=False, na=False)]['gameId'].values
    return game_ids[0] if len(game_ids) > 0 else None


In [43]:
# # функція для пошуку gameId за назвою

# # функція для отримання рекомендацій за списком gameId
# def get_recommendations(game_ids, similarity_table, game_data):
#     # Переконайтеся, що всі game_ids є цілими числами
#     game_ids = [int(game_id) for game_id in game_ids if game_id is not None]

#     similar_users = similarity_table[:, game_ids].mean(axis=1)
#     most_similar_users = similar_users.argsort()[-5:][::-1]

#     recommendations = []
#     for user_id in most_similar_users:
#         recommended_games = game_data[game_data['userId'] == user_id].sort_values('rating', ascending=False)['title'].values
#         recommendations.extend(recommended_games)

#     # Видалення дублікатів і повернення результату
#     return list(dict.fromkeys(recommendations))

# # Приклад використання
# input_titles = ['Grand Theft Auto V', 'Fallout 4','S.T.A.L.K.E.R.: Shadow of Chernobyl']
# input_game_ids = [get_gameId_by_title(title, game_data) for title in input_titles]
# recommended_titles = get_recommendations(input_game_ids, similarity_table, game_data)

# print(recommended_titles)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Функція для отримання рекомендацій за списком gameId з використанням kNN
def get_recommendations_knn(game_ids, pivot_table, game_data, n_neighbors=5):
    # Переконайтеся, що всі game_ids є цілими числами
    game_ids = [int(game_id) for game_id in game_ids if game_id is not None]
    # Знаходження k найближчих сусідів для кожного gameId
    print(game_ids)
    distances, indices = knn.kneighbors(pivot_table.loc[game_ids].values)
    # distances, indices = knn.kneighbors(pivot_table.loc[:, game_ids].values)
    # Отримання рекомендацій з використанням індексів найближчих сусідів
    recommendations = []
    for i in range(len(indices)):
        similar_users = indices[i]
        for user_id in similar_users:
            recommended_games = game_data[game_data['userId'] == user_id].sort_values('rating', ascending=False)['title'].values
            recommendations.extend(recommended_games)

    # Видалення дублікатів і повернення результату
    return list(dict.fromkeys(recommendations))

# Приклад використання
input_titles = ['Grand Theft Auto V', 'Fallout 4','S.T.A.L.K.E.R.: Shadow of Chernobyl','Відьмак® 3: Дикий гін', 'Remnant II','Subnautica: Below Zero','Counter-Strike: Condition Zero']
input_game_ids = [get_gameId_by_title(title, game_data) for title in input_titles]
recommended_titles_knn = get_recommendations_knn(input_game_ids, pivot_table, game_data)

print(recommended_titles_knn)


In [ ]:
import joblib
joblib.dump(knn, 'knn_model.pkl')

# Збереження матриці схожості
joblib.dump(similarity_table, 'similarity_table_2.pkl')

# Збереження pivot_table
pivot_table.to_pickle('pivot_table.pkl')

In [ ]:
game_data.head()

In [ ]:
game_data.drop(columns=['genres', 'steam_gameId', "id"], inplace=True)


In [ ]:
game_data.to_csv('user_data.csv', index=False)